In [90]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import os
from splinter import Browser
import time

In [91]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [92]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)
    
html = browser.html
soup = bs(html, "html.parser")
    
article = soup.find('div', class_='content_title')

In [93]:
print(article)

<div class="content_title"><a href="/news/8551/mars-scientists-investigate-ancient-life-in-australia/" target="_self">Mars Scientists Investigate Ancient Life in Australia</a></div>


In [94]:
#Get the Article Headline
headline = article.text
print(headline)

Mars Scientists Investigate Ancient Life in Australia


In [95]:
#Get the article text
article_text = soup.find('div', class_='article_teaser_body')
article_teaser = article_text.text
article_teaser

"Teams with NASA's Mars 2020 and ESA's ExoMars practiced hunting for fossilized microbial life in the Australian Outback in preparation for their Red Planet missions. "

In [96]:
#Get the most recent image
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url)

browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(3)

browser.click_link_by_partial_text('more info')

html = browser.html
image_soup = bs(html, 'html.parser')

featured_image_url = image_soup.find('figure', class_='lede').a['href']
featured_image_url = print(f'https://www.jpl.nasa.gov{featured_image_url}')
featured_image_url

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19046_hires.jpg


In [97]:
#Get the most recent tweet and take the weather from it.
mars_weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(mars_weather_url)
time.sleep(3)

html = browser.html
tweet_soup = bs(html, 'html.parser')


tweet_container = tweet_soup.find_all('div', class_="js-tweet-text-container")


for tweet in tweet_container: 
    mars_weather = tweet.find('p').text
    if 'sol' and 'pressure' in mars_weather:
        print(mars_weather)
        break
    else: 
        pass

InSight sol 343 (2019-11-13) low -100.8ºC (-149.4ºF) high -23.1ºC (-9.5ºF)
winds from the SSE at 5.2 m/s (11.7 mph) gusting to 20.8 m/s (46.5 mph)
pressure at 6.90 hPa


In [115]:
#Set up website to scrape.
mars_facts = 'https://space-facts.com/mars/'
browser.visit(mars_facts)

html = browser.html
facts_soup = bs(html, 'html.parser')

#Find the Table
table = pd.read_html(mars_facts)
mars_facts_table = table[0]

mars_facts_table

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [117]:
#Get the title and image from the Mars Hemisphere site.
mars_hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(mars_hemisphere_url)

html = browser.html
hemisphere_soup = bs(html, 'html.parser')

hemisphere_image_urls = []

results = hemisphere_soup.find('div', class_ = 'result-list')
hemispheres = results.find_all('div', class_='item')

for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    title = title.replace("Enhanced", "")
    end_link = hemisphere.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + end_link    
    browser.visit(image_link)
    html = browser.html
    soup = bs(html, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    hemisphere_image_urls.append({"title": title, "img_url": image_url})
    
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
